## TOPIC MODELING Using Gensim 

In [3]:
import gensim
import re
import pyLDAvis.sklearn
import pyLDAvis.gensim
from gensim.models import LdaModel , LdaMulticore,LsiModel
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
import nltk
from sklearn.decomposition import NMF
from nltk.corpus import stopwords

C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
stop_words = stopwords.words('english')
stop_words = stop_words + ['also','year','new','person','many','first']

In [5]:
### Add some to stop words if wanted after the result to neglet those from the distribution.

In [6]:
print(api.info("text8"))
dataset = api.load("text8")
data = [d for d in dataset]

{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [18]:
print(" ".join(data[2][:300]))

with the aegis of zeus when he goes to the battlefield the entire trojan army flees behind the walls of troy achilles wrath is terrible and he slays many trojan warriors and allies including priam s son lycaon whom achilles had previously captured and sold into slavery but who had been returned to troy eventually hector comes out of the walls to defend the honour of troy he asked achilles to agree that the body of the loser would be returned for proper burial by the winner achilles rejected this arrangement saying though twenty ransoms and thy weight in gold were offered i would refuse it all stories tell that hector ran about troy seven times and achilles followed him however seeing that achilles would not be outrun hector stood his ground and fought other versions of the tale say that achilles chased after hector two times and one time he was delivered by the gods however on their second encounter achilles trapped hector and challenged him after a legendary fight achilles kills hecto

In [7]:
## Preprocessing Data , lemmatizing , splitting each word and deleting words from stopword
data_processed = []
for i,doc in enumerate(data[:100]):
    doc_out=[]
    for x in doc:
        if x not in stop_words:
            #using only noun ,verb, adjectives and adverbs
            lemma = lemmatize(x,allowed_tags=re.compile('(NN|JJ|RB)'))
            if lemma:
                doc_out = doc_out + [lemma[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
#             data_processed.append(doc_out) doc_out , splitting and lemmatization
    data_processed.append(doc_out)

            

C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:625: DeprecationWarning: generator '_read' raised StopIteration
  dict.update(self, (x.split(" ")[:2] for x in _read(self._path) if len(x.split(" ")) > 1))
C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:959: DeprecationWarning: generator '_read' raised StopIteration
  for x in _read(self.path):
C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:2127: DeprecationWarning: generator '_read' raised StopIteration
  for v in _read(self._path):


In [8]:
## making a dictionary
new_dictionary =  corpora.Dictionary(data_processed)
dct = corpora.Dictionary(data_processed)


In [9]:
corpus = [dct.doc2bow(line) for line in data_processed]

In [14]:
## Lda model 
lda_model = LdaModel(corpus=corpus,id2word=dct,random_state=100,num_topics=7,passes=10,chunksize=1000,alpha='asymmetric',
                         decay=0.5,offset=64,eta=None,eval_every=0,iterations=100,gamma_threshold=0.001,per_word_topics=True)


In [15]:
lda_model.save('lda_model.model')

In [16]:
lda_model.print_topics(-1)

[(0,
  '0.006*"american" + 0.004*"state" + 0.003*"time" + 0.003*"world" + 0.003*"war" + 0.003*"person" + 0.002*"day" + 0.002*"year" + 0.002*"century" + 0.002*"name"'),
 (1,
  '0.003*"atheism" + 0.003*"god" + 0.003*"time" + 0.002*"asteroid" + 0.002*"atom" + 0.002*"number" + 0.002*"state" + 0.002*"world" + 0.002*"form" + 0.002*"albatross"'),
 (2,
  '0.004*"africa" + 0.003*"angola" + 0.003*"alberta" + 0.003*"acid" + 0.003*"abraham" + 0.002*"anchorage" + 0.002*"ascii" + 0.002*"north" + 0.002*"african" + 0.002*"province"'),
 (3,
  '0.007*"apollo" + 0.003*"lunar" + 0.003*"moon" + 0.002*"mission" + 0.002*"crew" + 0.002*"flight" + 0.002*"module" + 0.002*"spacecraft" + 0.002*"orbit" + 0.002*"earth"'),
 (4,
  '0.001*"american" + 0.000*"time" + 0.000*"world" + 0.000*"state" + 0.000*"war" + 0.000*"year" + 0.000*"number" + 0.000*"history" + 0.000*"language" + 0.000*"day"'),
 (5,
  '0.001*"american" + 0.001*"state" + 0.001*"jews" + 0.001*"adelaide" + 0.001*"war" + 0.001*"world" + 0.001*"person" + 0.

In [ ]:
#These topics can be classified into:--
    # topic 0,6 are more over same , about history and war stuff
    # topic 3 is fully based on apollo moon mission

In [19]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model,corpus,dct)
panel

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      41.802864        1       1  0.102203  0.039302
6      40.475971        1       2  0.102423  0.000155
1      14.043503        1       3  0.048084 -0.065371
2       2.479679        1       4 -0.043738  0.025740
3       1.193324        1       5 -0.058513 -0.004509
4       0.002492        1       6 -0.096433 -0.005321
5       0.002166        1       7 -0.054026  0.010004, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
60     Default  1614.000000      american  1614.000000  30.0000  30.0000
1419   Default  1549.000000         state  1549.000000  29.0000  29.0000
1628   Default  1009.000000           war  1009.000000  28.0000  28.0000
5718   Default   262.000000          jews   262.000000  27.0000  27.0000
1658   Default  1037.000000         world  1037.000000  26.0000  26.0000
1516   Default  1232.000000          time  1232.000000  25.0000  25.0000
1118   Default   880.000000        person   880.000000  24.0000  24.0000
7761   Default   150.000000      adelaide   150.000000  23.0000  23.0000
12688  Default   467.000000         agave   467.000000  22.0000  22.0000
2740   Default   530.000000         south   530.000000  21.0000  21.0000
1029   Default   743.000000        number   743.000000  20.0000  20.0000
211    Default   685.000000       century   685.000000  19.0000  19.0000
1665   Default   726.000000          year   726.000000  18.0000  18.0000
7982   Default   245.000000      football   245.000000  17.0000  17.0000
1021   Default   468.000000         north   468.000000  16.0000  16.0000
239    Default   645.000000          city   645.000000  15.0000  15.0000
702    Default   784.000000       history   784.000000  14.0000  14.0000
981    Default   762.000000          name   762.000000  13.0000  13.0000
841    Default   712.000000      language   712.000000  12.0000  12.0000
1759   Default   171.000000        apollo   171.000000  11.0000  11.0000
365    Default   686.000000           day   686.000000  10.0000  10.0000
1587   Default   682.000000           use   682.000000   9.0000   9.0000
668    Default   601.000000         group   601.000000   8.0000   8.0000
654    Default   529.000000           god   529.000000   7.0000   7.0000
1477   Default   712.000000        system   712.000000   6.0000   6.0000
602    Default   635.000000          form   635.000000   5.0000   5.0000
870    Default   467.000000          line   467.000000   4.0000   4.0000
20395  Default   127.000000          audi   127.000000   3.0000   3.0000
4735   Default   392.000000     australia   392.000000   2.0000   2.0000
5031   Default   231.000000        jewish   231.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
6234    Topic7     0.002671        engine   164.953568  -0.2911  -8.1305
60      Topic7     0.009133      american  1614.857910  -1.3429  -6.9010
1419    Topic7     0.008413         state  1549.807983  -1.3839  -6.9830
10961   Topic7     0.001181         wheel    41.117077   0.2822  -8.9464
8115    Topic7     0.000995     melbourne    31.037367   0.3920  -9.1179
1628    Topic7     0.006415           war  1009.329895  -1.2262  -7.2543
3903    Topic7     0.002484          ball   172.861130  -0.4103  -8.2028
2740    Topic7     0.004455         south   530.422180  -0.9475  -7.6189
28317   Topic7     0.000809  antisemitism    21.337694   0.5596  -9.3250
1658    Topic7     0.006093         world  1037.030273  -1.3047  -7.3057
1118    Topic7     0.005438        person   880.184204  -1.2545  -7.4194
201     Topic7     0.002440           car   185.605789  -0.4996  -8.2210
211     Topic7     0.004605       century   685.514099  -1.1709  -7.5858
1138    Topic7     0.003016          play   317.447296  -0.8243  -8.0090
870     Topic7     0.003637          lin

In [10]:
# We will try using LSI model - Latent semantic indexing

In [13]:
lsi_model = LsiModel(corpus=corpus, id2word=dct, num_topics=7, decay=0.5)

In [14]:
lsi_model.print_topics(-1)

[(0,
  '0.218*"american" + 0.217*"state" + 0.161*"time" + 0.141*"world" + 0.137*"war" + 0.117*"person" + 0.111*"agave" + 0.108*"history" + 0.100*"number" + 0.099*"language"'),
 (1,
  '0.935*"agave" + 0.163*"asia" + 0.100*"aruba" + 0.062*"plant" + 0.053*"var" + -0.048*"american" + 0.044*"state" + 0.044*"east" + 0.043*"congress" + 0.041*"maguey"'),
 (2,
  '0.501*"american" + 0.195*"football" + 0.189*"player" + 0.164*"war" + 0.153*"british" + 0.126*"ball" + -0.113*"god" + 0.111*"play" + -0.110*"atheism" + 0.110*"line"'),
 (3,
  '-0.329*"apollo" + 0.229*"lincoln" + 0.201*"state" + -0.191*"player" + -0.174*"football" + -0.148*"ball" + -0.147*"line" + -0.128*"play" + 0.127*"government" + 0.126*"union"'),
 (4,
  '-0.363*"atheism" + -0.344*"lincoln" + -0.334*"god" + -0.215*"atheist" + -0.212*"apollo" + -0.147*"abraham" + -0.136*"aristotle" + 0.134*"island" + 0.123*"aluminium" + -0.120*"belief"'),
 (5,
  '0.426*"lincoln" + -0.360*"atheism" + -0.269*"god" + 0.239*"apollo" + -0.214*"atheist" + -0

In [22]:
# only using Noun words this time

In [23]:
data_processed = []
for i,doc in enumerate(data[:50]):
    doc_out=[]
    for x in doc:
        if x not in stop_words:
            lemma = lemmatize(x,allowed_tags=re.compile('(NN)'))
            if lemma:
                doc_out = doc_out + [lemma[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
    data_processed.append(doc_out)
### 



In [24]:
dct = corpora.Dictionary(data_processed)
corpus = [dct.doc2bow(line) for line in data_processed]

In [25]:
lda_model = LdaModel(corpus=corpus,id2word=dct,random_state=100,num_topics=7,passes=10,chunksize=1000,alpha='asymmetric',
                         decay=0.5,offset=64,eta=None,eval_every=0,iterations=500,gamma_threshold=0.001,per_word_topics=True)

In [26]:
lda_model.print_topics(-1)

[(0,
  '0.005*"apple" + 0.005*"abortion" + 0.004*"time" + 0.004*"lincoln" + 0.004*"warhol" + 0.004*"art" + 0.004*"aristotle" + 0.003*"film" + 0.003*"war" + 0.003*"work"'),
 (1,
  '0.011*"anarchist" + 0.008*"anarchism" + 0.004*"autism" + 0.003*"movement" + 0.002*"child" + 0.001*"person" + 0.001*"proudhon" + 0.001*"anarchy" + 0.001*"society" + 0.001*"capitalism"'),
 (2,
  '0.008*"state" + 0.004*"world" + 0.004*"god" + 0.004*"war" + 0.004*"time" + 0.003*"atheism" + 0.003*"person" + 0.003*"country" + 0.003*"government" + 0.003*"language"'),
 (3,
  '0.001*"time" + 0.001*"language" + 0.001*"state" + 0.000*"africa" + 0.000*"line" + 0.000*"war" + 0.000*"number" + 0.000*"alphabet" + 0.000*"space" + 0.000*"word"'),
 (4,
  '0.006*"addiction" + 0.004*"anxiety" + 0.003*"axiom" + 0.002*"milne" + 0.002*"drug" + 0.002*"dependency" + 0.002*"phi" + 0.001*"pooh" + 0.001*"doyle" + 0.001*"substance"'),
 (5,
  '0.001*"aluminium" + 0.001*"island" + 0.001*"state" + 0.001*"acid" + 0.001*"lincoln" + 0.000*"war"

In [33]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model,corpus,dct)
pyLDAvis.display(panel)